In [ ]:
from crossword_solver.candidate_search_helpers import Candidate, search_candidates
from crossword_solver.crossword_helpers import Crossword, load_crossword
from crossword_solver.solver_helpers import solving_algorithm, display_results
import pandas as pd

In [ ]:
# Read in WordNet contents
wn_voc = pd.read_parquet('../data/wordnet_contents.parquet')

In [ ]:
# Read in crossword
cw = load_crossword('../data/crossword1.pickle')

In [ ]:
# Search for candidates
for i in range(len(cw.hints)):
    cw.hints[i].candidates = search_candidates(cw.hints[i])

In [ ]:
# Filter candidates with the right length
for i in range(len(cw.hints)):
    filtered = []
    for c in cw.hints[i].candidates:
        if len(c.text) == cw.hints[i].length:
            filtered.append(c)
    cw.hints[i].candidates = filtered

In [ ]:
"""
cw.hints[0].candidates.append(Candidate("kaamel", "mina", 1))
cw.hints[1].candidates.append(Candidate("sebra", "mina", 1))
cw.hints[2].candidates.append(Candidate("ninasarvik", "mina", 1))
cw.hints[3].candidates.append(Candidate("gepard", "mina", 1))
"""

In [ ]:
from copy import deepcopy
from crossword_solver.solver_helpers import find_suitable_candidates

def solving_algorithm(crossword):
    
    if len(crossword.hints)==0:
        return [crossword]
    
    hint = crossword.hints[0]
    suitable_candidates = find_suitable_candidates(hint, crossword)
    if(len(crossword.hints) > 2):
        print(len(crossword.hints), len(suitable_candidates))
        print(crossword)
    results = []
    new_crossword = deepcopy(crossword)
    new_crossword.hints.pop(0)
    result = solving_algorithm(new_crossword)
    results.extend(result)
    for candidate in suitable_candidates:
        new_crossword = deepcopy(crossword)
        for i, (x, y) in enumerate(hint.coordinates):
            new_crossword.matrix[x, y] = candidate.text[i]
        new_crossword.hints.pop(0)
        new_crossword.score += candidate.weight
        result = solving_algorithm(new_crossword)
        results.extend(result)
    return results

In [ ]:
def display_results(solving_algorithm_results, topn = 20):
    sorted_results = sorted(solving_algorithm_results, key = lambda x:x.score)[::-1]
    for solution in sorted_results[:topn]:
        print(solution.score)
        print(solution)
    return

In [ ]:
results = solving_algorithm(cw)
display_results(results)